In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
import re

import projects_parser

In [21]:
BOOKLET_FILE = "../booklets_text/2011.txt"

In [22]:
with open(BOOKLET_FILE, encoding="utf-8") as f:
    lines = f.readlines()

In [23]:
print(lines[0:20])
print(len(lines))

['Project Title: TMT Project (Take Me There) - MultiModal National\n', 'Public Transport Information System\n', 'Name: Ovidiu Bernaschi\n', 'Email: bernaso2@computing.dcu.ie\n', 'Programme: Information Systems\n', 'Supervisor: Dr. Martin Crane\n', 'TMT Project aims to deliver a public Information System which would generate on request a national\n', 'Multi-Modal Public Transport Travel Itinerary that will guide commuters from their point of origin to\n', 'their desired destination within the Republic of Ireland.\n', 'It aims to integrate as many Irish public transport companies as possible, then offer routing and access\n', 'to this public IS from all web-compatible devices like smartphones, tabletPCs, laptops, desktops etc\n', 'TMT Project received collaboration & support from: Department of Transport Ireland; Former Minister\n', 'for Transport Mr Noel Dempsey TD; National Transport Authority; Transport 21 offi ce; Dublin Chamber\n', 'of Commerce; Irish Rail, Eirebus/Urbus and Dublin 

In [24]:
# get rid of project numbering
filter_pattern = re.compile(r'^Project Number', re.M)
lines_filtered = list(filter(lambda line: not filter_pattern.match(line), lines))

In [25]:
print(lines_filtered[0:30])
print(len(lines_filtered))

['Project Title: TMT Project (Take Me There) - MultiModal National\n', 'Public Transport Information System\n', 'Name: Ovidiu Bernaschi\n', 'Email: bernaso2@computing.dcu.ie\n', 'Programme: Information Systems\n', 'Supervisor: Dr. Martin Crane\n', 'TMT Project aims to deliver a public Information System which would generate on request a national\n', 'Multi-Modal Public Transport Travel Itinerary that will guide commuters from their point of origin to\n', 'their desired destination within the Republic of Ireland.\n', 'It aims to integrate as many Irish public transport companies as possible, then offer routing and access\n', 'to this public IS from all web-compatible devices like smartphones, tabletPCs, laptops, desktops etc\n', 'TMT Project received collaboration & support from: Department of Transport Ireland; Former Minister\n', 'for Transport Mr Noel Dempsey TD; National Transport Authority; Transport 21 offi ce; Dublin Chamber\n', 'of Commerce; Irish Rail, Eirebus/Urbus and Dublin 

In [26]:
# Separate lines into list of projects

project_title_indices = [i for i, line in enumerate(lines_filtered) if "Project Title" in line]
print(project_title_indices[0:5])

projects_strings = []
i = 1
while i < len(project_title_indices):
    projects_strings.append("".join(lines_filtered[project_title_indices[i-1]:project_title_indices[i]]))
    i += 1
projects_strings.append("".join(lines_filtered[project_title_indices[-1]:]))

[0, 23, 47, 75, 99]


In [27]:
print(projects_strings[0:3])
print(len(projects_strings))

['Project Title: TMT Project (Take Me There) - MultiModal National\nPublic Transport Information System\nName: Ovidiu Bernaschi\nEmail: bernaso2@computing.dcu.ie\nProgramme: Information Systems\nSupervisor: Dr. Martin Crane\nTMT Project aims to deliver a public Information System which would generate on request a national\nMulti-Modal Public Transport Travel Itinerary that will guide commuters from their point of origin to\ntheir desired destination within the Republic of Ireland.\nIt aims to integrate as many Irish public transport companies as possible, then offer routing and access\nto this public IS from all web-compatible devices like smartphones, tabletPCs, laptops, desktops etc\nTMT Project received collaboration & support from: Department of Transport Ireland; Former Minister\nfor Transport Mr Noel Dempsey TD; National Transport Authority; Transport 21 offi ce; Dublin Chamber\nof Commerce; Irish Rail, Eirebus/Urbus and Dublin Bikes; IBM Ireland; DCU.\nPresentation website: www.

In [28]:
regex_schema = {
    'title': r'Project Title:(.*?)Name',
    'students': r'Name:(.*?)Programme',
    'programme': r'Programme:(.*)Supervisor',
    'supervisor_and_description': r'Supervisor:(.*?)(?:Primary Area:|Secondary Area:|Primary OS:|Primary Technology:|Secondary Technology:|$)',
    'primary_area': r'Primary Area:(.*?)(?:Secondary Area:|Primary OS:|Primary Technology:|Secondary Technology:|$)',
    'secondary_area': r'Secondary Area:(.*?)(?:Primary OS:|Primary Technology:|Secondary Technology:|$)',
    'primary_os': r'Primary OS:(.*?)(?:Primary Technology:|Secondary Technology:|$)',
    'primary_tech': r'Primary Technology:(.*?)(?:Secondary Technology|$)',
    'secondary_tech': r'Secondary Technology:(.*?)$',
}

In [29]:
projects = projects_parser.parse_projects(projects_strings, regex_schema)

In [31]:
projects[0:3]

[{'title': ' TMT Project (Take Me There) - MultiModal National\nPublic Transport Information System\n',
  'students': ' Ovidiu Bernaschi\nEmail: bernaso2@computing.dcu.ie\n',
  'programme': ' Information Systems\n',
  'supervisor_and_description': ' Dr. Martin Crane\nTMT Project aims to deliver a public Information System which would generate on request a national\nMulti-Modal Public Transport Travel Itinerary that will guide commuters from their point of origin to\ntheir desired destination within the Republic of Ireland.\nIt aims to integrate as many Irish public transport companies as possible, then offer routing and access\nto this public IS from all web-compatible devices like smartphones, tabletPCs, laptops, desktops etc\nTMT Project received collaboration & support from: Department of Transport Ireland; Former Minister\nfor Transport Mr Noel Dempsey TD; National Transport Authority; Transport 21 offi ce; Dublin Chamber\nof Commerce; Irish Rail, Eirebus/Urbus and Dublin Bikes; IB

In [32]:
projects_final = projects_parser.canonicalize_projects(projects, name_sep='Name:', email_sep='Email:')

In [33]:
projects_final[0:3]

[{'title': 'TMT Project (Take Me There) - MultiModal National Public Transport Information System',
  'students': [{'name': 'Ovidiu Bernaschi',
    'email': 'bernaso2@computing.dcu.ie'}],
  'programme': 'Information Systems',
  'primary_area': 'Web Application',
  'secondary_area': 'Navigation Information Platform',
  'primary_os': 'Multi-platform',
  'primary_tech': 'Google Maps Javascript API V3',
  'secondary_tech': 'Sencha Ext JS, MySQL, PHP',
  'supervisor': 'Dr. Martin Crane',
  'description': 'TMT Project aims to deliver a public Information System which would generate on request a national Multi-Modal Public Transport Travel Itinerary that will guide commuters from their point of origin to their desired destination within the Republic of Ireland. It aims to integrate as many Irish public transport companies as possible, then offer routing and access to this public IS from all web-compatible devices like smartphones, tabletPCs, laptops, desktops etc TMT Project received collabor

In [34]:
len(projects_final)

69

In [35]:
projects_parser.write_json('2011', projects_final)